In [1]:
import pandas as pd

In [2]:
schools_file = "Resources/schools_complete.csv"
schools_df = pd.read_csv(schools_file)

students_file ="Resources/students_complete.csv"
students_df = pd.read_csv(students_file)

merge_df = pd.merge(students_df, schools_df, how="left", on=["school_name","school_name"])
merge_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
students_df.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')

In [4]:
#Calculate the total number of schools
num_schools = merge_df["school_name"].nunique()
#Calculate the total number of students with ","
num_students = merge_df["Student ID"].nunique()
#Calculate the total budget in dollars
dist_budget = schools_df["budget"].sum()
#Calculate the average math score 
avg_score_math = merge_df["math_score"].mean()
#Calculate the average reading score
avg_score_read = merge_df["reading_score"].mean()
#Calculate the percentage of students with a passing math score (70 or greater)
tot_pass_math = merge_df["math_score"][merge_df.loc[:,"math_score"] >= 70].count()
pass_math = (tot_pass_math)/(merge_df["math_score"].count())*100
#Calculate the percentage of students with a passing reading score (70 or greater)
tot_pass_read = merge_df["reading_score"][merge_df.loc[:,"reading_score"] >= 70].count()
pass_read = (tot_pass_read)/(merge_df["reading_score"].count())*100
#Calculate the percentage of students who passed math **and** reading (% Overall Passing)
pass_both = merge_df[(merge_df["math_score"]>= 70) & (merge_df["reading_score"] >= 70)]
pass_both_group = len(pass_both)/num_students*100

In [5]:
#Create chart with info
dist_dict = {"Total Schools": [num_schools],
             "Total Students":[num_students],"Total Budget":[dist_budget],
            "Average Math Score":[avg_score_math],
             "Average Reading Score":[avg_score_read],"% Passing Math":[pass_math],
             "% Passing Reading":[pass_read], "% Overall Passing": [pass_both_group]}
dist_df=pd.DataFrame(data=dist_dict)
# dist_df["Total Budget"]=dist_df["Total Budget"].map("${:,.2f}".format)
# dist_df["Total Students"]=dist_df["Total Students"].map("{:,}".format)
dist_df
#Total Students should also display with "," 

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [6]:
#Create an overview table that summarizes key metrics about each school, including:
# School Name
schools_sort_df = schools_df.sort_values("school_name", ascending = True)
sch_group = merge_df.groupby(["school_name"])

# Average Math Score
sch_math = sch_group["math_score"].mean()
# Average Reading Score
sch_read = sch_group["reading_score"].mean()
# % Passing Math
#https://www.knytes.com/Cleaning-School-Data-with-Pandas/
sch_stud_total = sch_group["Student ID"].count()
students_sort_df = students_df.sort_values("school_name", ascending = True)
stu_pass_math = merge_df[merge_df["math_score"] >= 70].groupby(["school_name"])
sch_pass_math = [(i/j)*100 for i,j in zip(stu_pass_math.math_score.count(),sch_stud_total)]
# % Passing Reading
stu_pass_read = merge_df[merge_df["reading_score"] >= 70].groupby(["school_name"])
sch_pass_read = [(i/j)*100 for i,j in zip(stu_pass_read.reading_score.count(),sch_stud_total)]
# Per Student Budget
stu_budget = schools_sort_df["budget"]/schools_sort_df["size"]
# % Overall Passing (The percentage of students that passed math **and** reading.)
#Assist re: groupby function from:
#https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=7189be806a9ac33a129b6d8476f07d84946666d0&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f65726d69617367656c6179652f70616e6461732d6368616c6c656e67652f373138396265383036613961633333613132396236643834373666303764383439343636363664302f5079436974795363686f6f6c732f2e6970796e625f636865636b706f696e74732f5079436974795363686f6f6c735f737461727465722d636865636b706f696e742e6970796e62&logged_in=false&nwo=ermiasgelaye%2Fpandas-challenge&path=PyCitySchools%2F.ipynb_checkpoints%2FPyCitySchools_starter-checkpoint.ipynb&platform=android&repository_id=269505066&repository_type=Repository&version=96
pass_both_sch = merge_df[(merge_df["math_score"]>= 70) & (merge_df["reading_score"] >= 70)].\
groupby('school_name')['Student ID'].count()/sch_stud_total*100


In [7]:
sch_dict = {"School Name":schools_sort_df["school_name"],"School Type":schools_sort_df["type"], 
            "Total Students":schools_sort_df["size"],
            "Total School Budget":schools_sort_df["budget"], "Per Student Budget":stu_budget}
sch_df = pd.DataFrame(data=sch_dict)
stu_budget = sch_df["Total School Budget"]/sch_df["Total Students"]

In [8]:
sch_dict_2 = {"Total Students":sch_stud_total,"Average Math Score":sch_math, 
              "Average Reading Score":sch_read, 
              "% Passing Math":sch_pass_math, "% Passing Reading":sch_pass_read, 
              "% Overall Passing":pass_both_sch}
sch_df_2 =pd.DataFrame(data=sch_dict_2)

In [35]:
sch_sum_df = pd.merge(sch_df,sch_df_2, how="left", on=["Total Students","Total Students"])
# format_sch_sum_df = sch_sum_df
# format_sch_sum_df["Total Students"]=format_sch_sum_df["Total Students"].map("{:,}".format)
# format_sch_sum_df["Total School Budget"]=format_sch_sum_df["Total School Budget"].map("${:,.2f}".format)
# format_sch_sum_df["Per Student Budget"]=format_sch_sum_df["Per Student Budget"].map("${:,.2f}".format)
sch_sum_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [10]:
#Sort and display the top five performing schools by % overall passing.
top_five = sch_sum_df.sort_values("% Overall Passing", ascending=False)
top_five.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [11]:
#Sort and display the five worst-performing schools by % overall passing.
bottom_five = sch_sum_df.sort_values("% Overall Passing", ascending=True)
bottom_five.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [12]:
#Create a table that lists the average MathScore for students of each grade level (9th, 10th, 11th, 12th) 
#at each school.
#Create a pandas series for each grade. Hint: use a conditional statement.
#Group each series by school

g_nine_df=students_df.loc[students_df['grade']== '9th'].groupby('school_name').mean()
g_ten_df=students_df.loc[students_df['grade']== '10th'].groupby('school_name').mean()
g_elev_df=students_df.loc[students_df['grade']== '11th'].groupby('school_name').mean()
g_twel_df=students_df.loc[students_df['grade']== '12th'].groupby('school_name').mean()

#Combine the series into a dataframe

pass_by_grade = {"9th Grade": g_nine_df["math_score"], "10th Grade": g_ten_df["math_score"], 
                 "11th Grade": g_elev_df["math_score"], "12th Grade":g_twel_df["math_score"]}
math_by_grade = pd.DataFrame(pass_by_grade, index=schools_sort_df["school_name"])
math_by_grade
#Optional: give the displayed data cleaner formatting

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [13]:
#Create a table that lists the average Reading Score for students of each grade level 
#(9th, 10th, 11th, 12th) at each school.
#Create a pandas series for each grade. Hint: use a conditional statement.
#Group each series by school
#Combine the series into a dataframe
#Optional: give the displayed data cleaner formatting
pass_by_gradeb = {"9th Grade": g_nine_df["reading_score"], "10th Grade": g_ten_df["reading_score"], 
                 "11th Grade": g_elev_df["reading_score"], "12th Grade":g_twel_df["reading_score"]}
read_by_grade = pd.DataFrame(pass_by_gradeb, index=schools_sort_df["school_name"])
read_by_grade


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [14]:
stu_budget.sort_values(0)

C:\Users\chris\AppData\Local\Temp\ipykernel_3884\4211646677.py:1: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  stu_budget.sort_values(0)


5     578.0
8     581.0
6     582.0
10    583.0
2     600.0
9     609.0
4     625.0
7     628.0
11    637.0
14    638.0
1     639.0
13    644.0
12    650.0
3     652.0
0     655.0
dtype: float64

In [33]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#Use 4 reasonable bins to group school spending. Include in the table each of the following:
sch_sum_a=sch_sum_df
bins = [0,585,630,645,680]
spend_level = [">$585","$585-630","$630-645","$645-680"]
sch_sum_a["Per Student Budget"] =pd.cut(sch_sum_a["Per Student Budget"], bins, labels=spend_level)
spend_group_a =sch_sum_a.groupby("Per Student Budget")
spend_group_a.count()
spend_group_a[["Average Math Score", "Average Reading Score", "% Passing Math",
                "% Passing Reading", "% Overall Passing"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget,,,,,
>$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [34]:
#Create a table that breaks down school performances based on school size. 
#Use 4 reasonable bins to group school size. Include in the table each of the following:
sch_sum_b =sch_sum_df
bins = [0,1000,2000,5000]
sch_size = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
sch_sum_b["Total Students"] =pd.cut(sch_sum_b["Total Students"], bins, labels=sch_size)
size_group_b =sch_sum_b.groupby("Total Students")
size_group_b.count()
size_group_b[["Average Math Score", "Average Reading Score", "% Passing Math",
                "% Passing Reading", "% Overall Passing"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [51]:
#Create a table that breaks down school performances based on school type. 
#https://stackoverflow.com/questions/17702272/convert-pandas-series-containing-string-to-boolean
#replace({'C2': {'X': True, 'Y': False}})
sch_sum_c = sch_sum_df.replace({"School Type":{'Charter': 1, 'District': 2}})
bins = [0,1,2]
sch_type = ["Charter","District"]
sch_sum_c["School Type"] =pd.cut(sch_sum_c["School Type"], bins, labels=sch_type)
type_group_c =sch_sum_c.groupby("School Type")
type_group_c.count()
type_group_c[["Average Math Score", "Average Reading Score", "% Passing Math","% Passing Reading", "% Overall Passing"]].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
